# Fase 1: Importar las dependencias


In [1]:
import numpy as np
import math
import re                     #expresiones regulares
import pandas as pd
from bs4 import BeautifulSoup #para trabajar con txt en diferentes formatos


In [2]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
#esto es solamente en GoogleColab, es para cargar la última TF

from tensorflow.keras import layers
import tensorflow_datasets as tfds #de aquí tomaremos el 'tokenizador'

# Fase 2: Pre Procesado de Datos

## Carga de Ficheros

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Fuente del dataset: https://www.kaggle.com/clmentbisaillon/fake-and-real-news-dataset

In [4]:
t_news = pd.read_csv('/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Predicion fake news/True.csv')
f_news = pd.read_csv('/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Predicion fake news/Fake.csv')

print(t_news.shape)
print(f_news.shape)

total_nan_values_t = t_news.isnull().sum().sum()
print (total_nan_values_t)
total_nan_values_f = f_news.isnull().sum().sum()
print (total_nan_values_f)

(21417, 4)
(23481, 4)
0
0


In [5]:
t_news.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
f_news.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [7]:
#Uniremos la columna Titulo con la columna del Texto para no perder esa información

t_news["news"] = t_news["title"] + " " + t_news["text"]
t_news.drop(['title', 'text', 'subject', 'date'], axis=1, inplace=True)
t_etiqueta = t_news.assign(Etiqueta=1)
t_etiqueta.head(5)

,news,Etiqueta
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [8]:
f_news["news"] = f_news["title"] + " " + f_news["text"]
f_news.drop(['title', 'text', 'subject', 'date'], axis=1, inplace=True)
f_etiqueta = f_news.assign(Etiqueta=0)
f_etiqueta.head(5)

,news,Etiqueta
0,Donald Trump Sends Out Embarrassing New Year’...,0
1,Drunk Bragging Trump Staffer Started Russian ...,0
2,Sheriff David Clarke Becomes An Internet Joke...,0
3,Trump Is So Obsessed He Even Has Obama’s Name...,0
4,Pope Francis Just Called Out Donald Trump Dur...,0


In [9]:
news = pd.concat([t_etiqueta, f_etiqueta], axis=0)
news.reset_index(drop=True, inplace=True)
news

,news,Etiqueta
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1
...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,0
44896,How to Blow $700 Million: Al Jazeera America F...,0


El conjunto de datos de testing tiene 3 etiquetas diferentes (una negativa, una positiva y una neutra), mientras que el conjunto de datos de entrenamiento tiene solo dos, por lo que no usaremos el archivo de testing y dividiremos el archivo de entrenamiento más tarde nosotros mismos.

## Pre Procesado

### Limpieza

In [10]:
news.head(5)

,news,Etiqueta
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1


In [11]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    
    # Eliminamos la @ y su mención
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet) #https://regexr.com/ --> loquizar cualquier cosa que empiece con @+lo que sea y se sutituye por ' '

    # Eliminamos los links de las URLs
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet) #el ? significa que el caracter anterior es opcional
    
    # Nos quedamos solamente con los caracteres
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet) #eliminar todo menos va con el ^ y pongo en []
    
    # Eliminamos espacios en blanco adicionales
    tweet = re.sub(r" +", ' ', tweet)
    
    return tweet

In [12]:
data_clean = [clean_tweet(tweet) for tweet in news.news]

In [13]:
set(news.Etiqueta)

{0, 1}

In [14]:
data_clean

Output hidden; open in https://colab.research.google.com to view.

### Tokenización

In [15]:
#De una lista de caracteres pasaremos a una lista de números

tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    data_clean, target_vocab_size=2**16 #tamaño del vocabulario, en este casos las 65000 más frecuentes. Las menos frecuentes dificilmente la IA las pueda interpretar.
)

data_inputs = [tokenizer.encode(sentence) for sentence in data_clean]

### Padding

In [16]:
#Padding --> Agrega CEROS hasta igualar todas las frases en su extención

MAX_LEN = max([len(sentence) for sentence in data_inputs]) #max busca la frase más larga
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

### Dividimos en los conjuntos de entrenamiento y de testing


In [17]:
#Esto es porque los Tweet estan ordenados 800mil neg y 800mil pos, sacaremos 8000 al azar de la primera mitad y 8000 de la segunda mitad

test_idx = np.random.randint(0, 21417, 1000) #los random de los neg
test_idx = np.concatenate((test_idx, test_idx+21417)) # los pos se suman al mismo numero de los neg

In [18]:
data_labels = news.Etiqueta.values

In [19]:
test_inputs = data_inputs[test_idx] #txt tokenizados y padeado para entrenar la red neuronal
test_labels = data_labels[test_idx] #etiquetas 0 y 1 para entrenar la red neuronal
train_inputs = np.delete(data_inputs, test_idx, axis=0) #lo mismo pero le saco los utilizados arriba
train_labels = np.delete(data_labels, test_idx) #sin axis porque la etiqueta no es una matriz como lo son los txt

In [20]:
print(test_inputs.shape)
print(test_labels.shape)
print(train_inputs.shape)
print(train_labels.shape)

(2000, 8840)
(2000,)
(42932, 8840)
(42932,)


In [21]:
print(type(test_inputs))
print(type(test_labels))
print(type(train_inputs))
print(type(train_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# Fase 3: Construción del modelo

In [22]:
#Aquí comienza la CONVULSION, es decir, la aplicación de los filtros

class DCNN(tf.keras.Model): #instanciamos una clase
    
    def __init__(self,
                 vocab_size, #tamaño del vocab, dado por el tokenizador
                 emb_dim=128, #cada palabra será resumida a un numero de 128 caraceteres
                 nb_filters=50, #se pondrán 50 filtros de 4, 3 y 2 palabras
                 FFN_units=512, #número de neuronas de la capa oculta
                 nb_classes=2, #número de categorias de salida, en este caso son 2 por o neg
                 dropout_rate=0.1, #se usa para el entrenamiento, algunas neuronas se apagan para que no aprendan todas juntas, en cada vuelta el 10% se apagará 
                 training=False, #en la fase de predicción las neuronas funcionan todas
                 name="dcnn"): #le damos un nombre al modelo
        super(DCNN, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim) #tamaño del vocab
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2, #filtro de 2 palabras
                                    padding="valid",
                                    activation="relu") #cdo se activa la neurona? con esto que anda bien, es automático
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,#filtro de 3 palabras
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4, #filtro de 4 palabras
                                      padding="valid",
                                      activation="relu")
#El pooling comprimirá la info        
        self.pool = layers.GlobalMaxPool1D() # No tenemos variable de entrenamiento
                                             # así que podemos usar la misma capa 
                                             # para cada paso de pooling

        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate) #esto es para que no colapse la red
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")#para clasificacion binaria, si hubiera más categorias no podría usarse
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")#para pasar de 2 categorias/binaria a más de dos
    
    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)
        
        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)
        
        return output

# Paso 4: Aplicación

## Configuración

In [23]:
#concretizar todas las variables y parametros que se crearon arriba

VOCAB_SIZE = tokenizer.vocab_size # 65540

EMB_DIM = 200 #cada palabra se identificara con 200 cordenadas
NB_FILTERS = 100 #numero de filtros de la red
FFN_UNITS = 256 #numero de unidades de feet fowards en la capa oculta
NB_CLASSES = 2 #len(set(train_labels)) --> según las etiquetas de entrenamiento

DROPOUT_RATE = 0.2 #20% de las neuronas se apagan en el entrenamiento

BATCH_SIZE = 32 #se probará de 32 en 32 elementos = tweets
NB_EPOCHS = 3 # el data set se iterará 5 veces para mejorar la precisión

## Entrenamiento

In [24]:
#se creea el OBJETO Deep Convutional Neuronal Network
#se le pasa la configuración anterior

Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [25]:
#¿qué haremos cdo el proceso de clasificacion es bianario?

if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])

#si no es binario se resuelve de la siguiente manera:

else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [26]:
#Esto es para que si se corta la sesión de GoogleColab y no arrancar desde el principio
#SIRVE PARA TENSOR FLOW

checkpoint_path = "/content/drive/Othercomputers/Mi PC/02. Programacion/Proyectos/Predicion fake news/ckpt"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

In [ ]:
#Ajustar los datos

Dcnn.fit(train_inputs,
         train_labels,
         batch_size=BATCH_SIZE,
         epochs=NB_EPOCHS)
ckpt_manager.save()

Epoch 1/3
  88/1342 [>.............................] - ETA: 8:49 - loss: 8.8336e-07 - accuracy: 1.0000

## Evaluación

In [ ]:
#aqui aplica lo entrenado a news que no ha leido antes

results = Dcnn.evaluate(test_inputs, test_labels, batch_size=BATCH_SIZE)
print(results)

Aquí se prueba con Tweets "artificiales", con el entrenamiento anterior

In [ ]:
Dcnn(np.array([tokenizer.encode("")]), training=False).numpy()

In [ ]:
tokenizer.encode("You are so funny")